In [2]:
import cv2
import sep
from astroquery.astrometry_net import AstrometryNet
import numpy as np
import math
from astropy.io import fits
import astropy.units as u
from astropy.coordinates.sky_coordinate import SkyCoord
from astropy import coordinates as coords
from astropy.units import Quantity
from astroquery.gaia import Gaia
from astropy.table import Table, hstack
from astropy.wcs import WCS
from scipy.optimize import curve_fit
from astroquery.sdss import SDSS

In [3]:
API_key = 'kjyxfcglvpnuthca'

In [4]:
def init(data_filename):
    data_filename = data_filename
    tif = cv2.imread(data_filename)
    tif_dat = tif.astype(np.float32)
    b, g, r = cv2.split(tif_dat)
    comb_tif = b + g + r
    invdat = -comb_tif+np.min(comb_tif)+np.max(comb_tif)
    fits_file = fits.writeto("{}.fits".format(data_filename[:-4]), invdat, overwrite = True)
    return [data_filename, comb_tif, invdat, fits_file]
    
    

In [5]:
def astrometry(initd, API, ra_guess, dec_guess, radius):
    ast = AstrometryNet()
    ast.api_key = API
    solve1 = ast.solve_from_image("{}.fits".format(initd[0][:-4]), force_image_upload=True, center_ra=ra_guess, center_dec=dec_guess, radius = radius, ra_dec_units=('degree', 'degree'))
    fits.writeto("{}.fits".format(initd[0][:-4]), initd[2], solve1, overwrite = True)
    solve2 = ast.solve_from_image("{}.fits".format(initd[0][:-4]), force_image_upload=True, center_ra=ra_guess, center_dec=dec_guess, radius = radius, ra_dec_units=('degree', 'degree'))
    fits.writeto("{}.fits".format(initd[0][:-4]), initd[2], solve2, overwrite = True)
    fits_solve2 = fits.open("{}.fits".format(initd[0][:-4]))
    return solve2
    

In [6]:
def septrac(initd, astrom, sigma = 3.0, gain = 1.0):
    bkg = sep.Background(initd[2])
    wcssolve = WCS(astrom)
    bkg_sub = initd[2] - bkg
    objects = sep.extract(bkg_sub, sigma, err=bkg.globalrms)
    objtable = Table(objects)
    coord_p = wcssolve.pixel_to_world(objtable['x'], objtable['y'])
    objtable['ra_p'], objtable['dec_p'] = coord_p.ra.deg, coord_p.dec.deg
    objtable['pet_mag'] = 25 - 2.5*np.log10(objtable['flux'])
    return [objtable, wcssolve]



In [7]:
def gaia_call(initd, astrom, sept):
    mid_coord = sept[1].pixel_to_world(round(np.shape(initd[2])[0]/2), round(np.shape(initd[2])[1]/2))
    bottom_coord = sept[1].pixel_to_world(0, 0)
    top_coord = sept[1].pixel_to_world(np.shape(initd[2])[0], np.shape(initd[2])[1])
    full_coord_ra = np.abs(top_coord.ra.deg-bottom_coord.ra.deg)
    full_coord_dec = np.abs(top_coord.dec.deg-bottom_coord.dec.deg)
    job = Gaia.launch_job("SELECT TOP 500000 "
                    "source_id,ra,dec,parallax,parallax_error,pm,pmra,pmra_error,pmdec,pmdec_error,phot_g_mean_mag,"
                    "phot_bp_mean_mag,phot_bp_mean_flux_over_error,phot_rp_mean_mag,phot_rp_mean_flux_over_error,bp_rp,phot_variable_flag, classprob_dsc_combmod_galaxy"
                    " from gaiadr3.gaia_source"
                    " WHERE CONTAINS(POINT('ICRS',ra,dec),BOX('ICRS',{0},{1},{2},{3}))=1  AND  ((phot_bp_mean_mag + 1.1*bp_rp) <= 19.0)".format(mid_coord.ra.deg, mid_coord.dec.deg, full_coord_ra+0.3, full_coord_dec+0.3))
    gaia_res = job.get_results()
    return gaia_res

In [8]:
def match_cat(initd, astrom, sept, gaiac):
    coords_gaia = SkyCoord(gaiac['ra'], gaiac['dec'], frame = 'icrs', unit = 'deg')
    coords_plate = SkyCoord(sept[0]['ra_p'], sept[0]['dec_p'], frame = 'icrs', unit = 'deg')
    index, diff2, diff3 = coords_plate.match_to_catalog_sky(coords_gaia)
    match_cat = hstack([gaiac[index], sept[0]])
    match_cat = match_cat[np.unique(match_cat['source_id'], return_index = True)[1]]
    match_cat['ang_dist'] = np.abs(np.sqrt(match_cat['ra']**2+match_cat['dec']**2)-np.sqrt(match_cat['ra_p']**2+match_cat['dec_p']**2)) #i dont know if this makes any sense, distance formula
    match_cat['pg'] = match_cat['phot_bp_mean_mag']+0.9*match_cat['bp_rp']
    match_cat['dec_res'] = (match_cat['dec'] -  match_cat['dec_p'])*3600
    match_cat['ra_res'] = (match_cat['ra'] -  match_cat['ra_p'])*3600*0.9114
    return [match_cat, coords_gaia]

In [80]:
def transform(initd, astrom, sept, gaiac, matca):
        tabu = matca[0]
        grama = 16
        smalm = 14
        gal_choi = []
        while len(gal_choi) == 0:
            grama += 1
            smalm -= 1
            mag_up = tabu[tabu['pg'] < grama]
            mag_mid = mag_up[mag_up['pg'] > smalm]
            gal_choi = mag_mid[mag_mid['classprob_dsc_combmod_galaxy'] == 1]

        hdu1 = (astrom)
        imdata = initd[1]
        im_raa = np.arange(np.shape(initd[2])[0])
        im_deca = np.arange(np.shape(initd[2])[1])
        im_grra, im_grdec = np.meshgrid(im_raa, im_deca)
        im_vals = sept[1].pixel_to_world(im_grra, im_grdec)
        
        def log_x(x, a,b,c,d):
            x = (a/np.log10(c*x-b)+d)**2
            return x

        galies = []
        for galy in range(0,len(gal_choi)):
            pos = coords.SkyCoord(gal_choi['ra'][galy], gal_choi['dec'][galy], frame='icrs', unit=u.deg)
            image = SDSS.get_images(matches=Table(SDSS.query_region(pos)), band='g')
            sdat = image[0][0].data
            ra_pix = np.arange(image[0][0].header['NAXIS1'])
            dec_pix = np.arange(image[0][0].header['NAXIS2'])
            gridra, griddec = np.meshgrid(ra_pix, dec_pix)
            sdss_val = WCS(image[0][0].header).pixel_to_world(gridra, griddec)

            inn, di2, di3 = sdss_val.flatten().match_to_catalog_sky(im_vals.flatten())

            dat1 = imdata.flatten()[inn]
            gg1 = sdat.flatten()

            popts = []
            sortd = np.sort([dat1,gg1], 1)
            split = 2
            while True:
                try:
                    wanted = []
                    spl_so = np.array_split(sortd, split, 1)
                    for arr in range(0,split):
                        migval = np.mean(spl_so[arr][0])
                        s_vau = np.mean(spl_so[arr][1])
                        wanted.append([migval+0.0000001*arr, s_vau])
                    wanted.append([np.min(dat1), np.max(gg1)+100])
                    wanted.append([np.max(dat1), np.min(gg1)-20])
                    wanted = np.array(wanted)
                    shap = np.shape(wanted)
                    wanted = np.sort(np.reshape(wanted, (shap)), axis = 0)
                    popt, pcov = curve_fit(log_x, (wanted[:,0]), (wanted[:,1][::-1]), maxfev = 5000)
                    popts.append([split, popt, np.nansum(np.nansum(np.abs(imdata-log_x(imdata, *popt))**2))])
                except:
                    popts = np.array(popts, dtype = object)
                    break
                else:
                    split += 25
            popts = np.array(popts, dtype = object)
            if len(popts) == 0:
                pass
            else:
                rig = np.where(np.min((popts[:,2])) == (popts[:,2]))
                galies.append(popts[rig])
        galies = np.array(galies)
        finr = np.where(np.min(galies[:,0,2]) == (galies[:,0,2]))
        gcdata = log_x(imdata, *galies[finr[0][0]][0][1])
#         fits.writeto("{}_galcal.fits".format(initd[0][:-4]), gcdata, astrom, overwrite = True)
#         fits_galcal = fits.open("{}_galcal.fits".format(initd[0][:-4]))
        return(gcdata)

In [10]:
def gc_sep(initd, astrom, sept, gaiac, matca, trant, sigma = 3.0, gain = 1.0):
    bkg = sep.Background(trant)
    bkg_sub = trant - bkg
    objects = sep.extract(bkg_sub, sigma, err=bkg.globalrms)
    objtable = Table(objects)
    coord_p = sept[1].pixel_to_world(objtable['x'], objtable['y'])
    objtable['ra_p'], objtable['dec_p'] = coord_p.ra.deg, coord_p.dec.deg
    objtable['pet_mag'] = 25 - 2.5*np.log10(objtable['flux'])
    return objtable

In [11]:
def gc_match(initd, astrom, sept, gaiac, matca, sepgc):
    coords_plate = SkyCoord(sepgc['ra_p'], sepgc['dec_p'], frame = 'icrs', unit = 'deg')
    index, diff2, diff3 = coords_plate.match_to_catalog_sky(matca[1])
    gc_match = hstack([gaiac[index], sepgc])
    gc_match = gc_match[np.unique(gc_match['source_id'], return_index = True)[1]]
    gc_match['ang_dist'] = np.abs(np.sqrt(gc_match['ra']**2+gc_match['dec']**2)-np.sqrt(gc_match['ra_p']**2+gc_match['dec_p']**2)) #i dont know if this makes any sense, distance formula
    gc_match['pg'] = gc_match['phot_bp_mean_mag']+0.9*gc_match['bp_rp']
    gc_match['dec_res'] = (gc_match['dec'] -  gc_match['dec_p'])*3600
    gc_match['ra_res'] = (gc_match['ra'] -  gc_match['ra_p'])*3600*0.9114
    return(gc_match)

In [78]:
%%time
initd = init('/Users/irescapa/Downloads/R3170_det_pp.tif')
astrom = astrometry(initd,API_key, 257.5000, 43.7500, 10)
sept = septrac(initd, astrom, 5)
gaiac = gaia_call(initd, astrom, sept)
matca = match_cat(initd, astrom, sept, gaiac)

Solving.......Solving...............CPU times: user 3.28 s, sys: 1.84 s, total: 5.12 s
Wall time: 52.2 s


In [81]:
%%time
trant = transform(initd, astrom, sept, gaiac, matca)
sepgc = gc_sep(initd, astrom, sept, gaiac, matca, trant, sigma = 5, gain = 1.0)
fincat = gc_match(initd, astrom, sept, gaiac, matca, sepgc)

<ipython-input-80-102e08fb344b>:21: RuntimeWarning: invalid value encountered in log10
  x = (a/np.log10(c*x-b)+d)**2


CPU times: user 2min 48s, sys: 16.5 s, total: 3min 5s
Wall time: 3min 7s


In [82]:
fincat

source_id,ra,dec,parallax,parallax_error,pm,pmra,pmra_error,pmdec,pmdec_error,phot_g_mean_mag,phot_bp_mean_mag,phot_bp_mean_flux_over_error,phot_rp_mean_mag,phot_rp_mean_flux_over_error,bp_rp,phot_variable_flag,classprob_dsc_combmod_galaxy,thresh,npix,tnpix,xmin,xmax,ymin,ymax,x,y,x2,y2,xy,errx2,erry2,errxy,a,b,theta,cxx,cyy,cxy,cflux,flux,cpeak,peak,xcpeak,ycpeak,xpeak,ypeak,flag,ra_p,dec_p,pet_mag,ang_dist,pg,dec_res,ra_res
,deg,deg,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mag,mag,,mag,,mag,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,deg,mag,deg,deg
int64,float64,float64,float64,float32,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,object,float32,float64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float32,float64,float64
1360187662717292160,259.1292165741683,42.84329065810747,0.3786064788199535,0.030454269,6.384339,2.224177673025725,0.033747137,-5.984380831073483,0.037840497,16.143578,16.451704,341.15277,15.611775,391.67712,0.8399286,NOT_AVAILABLE,5.2170524e-13,0.001198393409140408,8,7,1226,1228,0,2,1226.871679272654,0.8771915218748364,0.5621857473594696,0.5496329485353042,-0.10331723715329633,0.0018361328381201357,0.001836114083277514,-0.00042373722268143545,0.8120449781417847,0.6726080775260925,-0.7550610303878784,1.8424190282821655,1.8844971656799316,0.6926573514938354,0.012350047007203102,0.015546578913927078,0.0020756882149726152,0.002778533613309264,1227,1,1227,1,2,259.1268190382338,42.84332365836514,29.520912910847706,0.002360040077405756,17.20764,-0.11880092759781746,7.866411302551132
1360182302597958656,258.86052662717714,42.70840480809556,0.8582376619225857,0.021039546,6.088845,2.2594175037885385,0.026064226,5.6541192589346565,0.026897041,15.025394,15.373041,552.1504,14.504373,813.70135,0.86866856,NOT_AVAILABLE,5.124686e-13,0.001198393409140408,11,11,784,789,4,7,786.5153825762573,5.614466431634561,2.0617052012948793,1.1428138231248854,1.2995283712216192,0.005835916257001067,0.0032213701570091213,0.0036418776876505723,1.7264457941055298,0.4731847941875458,0.6154820919036865,1.7124110460281372,3.0892930030822754,-3.8944692611694336,0.014960412867367268,0.017752189189195633,0.001453462173230946,0.0019745214376598597,787,6,784,4,0,258.86534556580193,42.72551359359162,29.376870205936026,0.007540216935410626,16.154842,-61.59162778582754,-15.811130385494435
1360187490918602752,259.0798031904234,42.80717928677785,0.4081947066681394,0.03853049,9.217796,-0.7354652609393404,0.045039795,-9.188408556356595,0.05080244,16.653845,16.964712,223.59898,16.184967,298.50598,0.7797451,NOT_AVAILABLE,5.125074e-13,0.001198393409140408,7,7,1141,1143,14,16,1141.6797798229463,15.006839221007871,0.43466464711690134,0.5444095072067991,-0.013980757309629,0.0014690221155125058,0.0017098429268759831,-7.060876331950337e-07,0.7390280961990356,0.6579601764678955,-1.4460570812225342,2.3025259971618652,1.8383710384368896,0.118260458111763,0.011593086645007133,0.014822607859969139,0.0020852694287896156,0.002866209950298071,1142,15,1142,15,0,259.0712360924725,42.8263026841642,29.572688451830246,0.005334266027432477,17.666483,-68.844230590841,28.108991060826924
1360405881415411328,259.31537231643375,43.190582552711724,0.10355893498499062,0.10085568,5.240124,-5.209891767753999,0.1259562,-0.5620753841959242,0.11133122,18.19669,18.280176,72.21269,17.642988,66.230934,0.63718796,NOT_AVAILABLE,0.0032326186,0.001198393409140408,119,106,2522,2532,7,20,2526.5464400581445,14.458497779084091,3.270945614293808,6.115244067811858,-1.1194445897003007,9.681219250270496e-06,1.7772186499816416e-05,-2.5323859963310384e-06,2.550092935562134,1.6980032920837402,-1.2373682260513306,0.32615548372268677,0.17445531487464905,0.11941076815128326,2.2966227531433105,2.311380624771118,0.08137239515781403,0.0972118228673935,2527,15,2527,15,0,259.89